# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [3]:
from typing import List, Set, Tuple, Optional
from dataclasses import dataclass
from queue import PriorityQueue
import copy

@dataclass
class State:
    left_bank: Set[str]  # People on left bank
    island: Set[str]     # People on island
    right_bank: Set[str] # People on right bank
    boat_location: str   # 'left', 'island', or 'right'
    
    def __str__(self):
        return f"Left: {self.left_bank}, Island: {self.island}, Right: {self.right_bank}, Boat: {self.boat_location}"

def is_valid_state(state: State, couples: List[Tuple[str, str]]) -> bool:
    """Check if the current state is valid according to the jealousy constraint."""
    def check_location(people: Set[str]) -> bool:
        for wife, husband in couples:
            if wife in people and husband not in people:
                # If a wife is present, check if any other husband is present
                for _, other_husband in couples:
                    if other_husband != husband and other_husband in people:
                        return False
        return True
    
    return (check_location(state.left_bank) and 
            check_location(state.island) and 
            check_location(state.right_bank))

def get_possible_moves(state: State) -> List[Tuple[Set[str], str]]:
    """Get all possible moves from current state."""
    current_location = state.boat_location
    if current_location == 'left':
        source = state.left_bank
        next_locations = ['island']
    elif current_location == 'island':
        source = state.left_bank | state.island
        next_locations = ['left', 'right']
    else:  # right
        source = state.right_bank
        next_locations = ['island']
    
    moves = []
    # Try moving 1 or 2 people
    for next_loc in next_locations:
        for person in source:
            moves.append(({person}, next_loc))
            for other_person in source - {person}:
                moves.append(({person, other_person}, next_loc))
    
    return moves

def apply_move(state: State, move: Tuple[Set[str], str]) -> State:
    """Apply a move to the current state and return new state."""
    people, next_location = move
    new_state = copy.deepcopy(state)
    
    # Remove people from current location
    if state.boat_location == 'left':
        new_state.left_bank -= people
    elif state.boat_location == 'island':
        new_state.island -= people
    else:
        new_state.right_bank -= people
    
    # Add people to next location
    if next_location == 'left':
        new_state.left_bank |= people
    elif next_location == 'island':
        new_state.island |= people
    else:
        new_state.right_bank |= people
    
    new_state.boat_location = next_location
    return new_state



# Create initial state
n_couples = 2
couples = [(f'H{i}', f'W{i}') for i in range(1, n_couples + 1)]
all_people = {person for couple in couples for person in couple}

initial_state = State(
    left_bank=all_people,
    island=set(),
    right_bank=set(),
    boat_location='left'
)


In [4]:
# 1. DFS Solution
def dfs_solve(initial_state: State, couples: List[Tuple[str, str]]) -> Optional[List[State]]:
    visited = set()
    path = []
    
    def dfs(current_state: State) -> bool:
        state_str = str(current_state)
        if state_str in visited:
            return False
        
        visited.add(state_str)
        path.append(current_state)
        
        # Check if goal state
        if not current_state.left_bank and not current_state.island:
            return True
        
        # Try all possible moves
        for move in get_possible_moves(current_state):
            new_state = apply_move(current_state, move)
            if is_valid_state(new_state, couples):
                if dfs(new_state):
                    return True
        
        path.pop()
        return False
    
    if dfs(initial_state):
        return path
    return None

print("Solving with DFS:")
dfs_solution = dfs_solve(initial_state, couples)
if dfs_solution:
    print(f"Found solution with {len(dfs_solution)} steps")
    for i, state in enumerate(dfs_solution):
        print(f"Step {i}: {state}")
else:
    print("No solution found with DFS")

Solving with DFS:
Found solution with 57 steps
Step 0: Left: {'H2', 'W2', 'H1', 'W1'}, Island: set(), Right: set(), Boat: left
Step 1: Left: {'W2', 'H1', 'W1'}, Island: {'H2'}, Right: set(), Boat: island
Step 2: Left: {'H2', 'W2', 'W1', 'H1'}, Island: set(), Right: set(), Boat: left
Step 3: Left: {'W1', 'H1'}, Island: {'H2', 'W2'}, Right: set(), Boat: island
Step 4: Left: {'W2', 'H1', 'W1'}, Island: {'H2'}, Right: set(), Boat: left
Step 5: Left: {'H1', 'W1'}, Island: {'H2', 'W2'}, Right: set(), Boat: island
Step 6: Left: {'H1', 'W1'}, Island: {'H2', 'W2'}, Right: set(), Boat: left
Step 7: Left: set(), Island: {'H2', 'W2', 'H1', 'W1'}, Right: set(), Boat: island
Step 8: Left: {'H2', 'W2'}, Island: {'W1', 'H1'}, Right: set(), Boat: left
Step 9: Left: {'H2'}, Island: {'W2', 'H1', 'W1'}, Right: set(), Boat: island
Step 10: Left: {'H2', 'H1'}, Island: {'W2', 'W1'}, Right: set(), Boat: left
Step 11: Left: {'H1'}, Island: {'H2', 'W2', 'W1'}, Right: set(), Boat: island
Step 12: Left: {'H1'}, I

In [5]:

# 2. Backtracking Solution
def backtracking_solve(initial_state: State, couples: List[Tuple[str, str]]) -> Optional[List[State]]:
    visited = set()
    path = []
    
    def backtrack(current_state: State) -> bool:
        state_str = str(current_state)
        if state_str in visited:
            return False
        
        visited.add(state_str)
        path.append(current_state)
        
        # Check if goal state
        if not current_state.left_bank and not current_state.island:
            return True
        
        # Try all possible moves
        for move in get_possible_moves(current_state):
            new_state = apply_move(current_state, move)
            if is_valid_state(new_state, couples):
                if backtrack(new_state):
                    return True
        
        # Backtrack
        path.pop()
        visited.remove(state_str)
        return False
    
    if backtrack(initial_state):
        return path
    return None

print("\nSolving with Backtracking:")
backtracking_solution = backtracking_solve(initial_state, couples)
if backtracking_solution:
    print(f"Found solution with {len(backtracking_solution)} steps")
    for i, state in enumerate(backtracking_solution):
        print(f"Step {i}: {state}")
else:
    print("No solution found with Backtracking")
    


Solving with Backtracking:
Found solution with 57 steps
Step 0: Left: {'H2', 'W2', 'H1', 'W1'}, Island: set(), Right: set(), Boat: left
Step 1: Left: {'W2', 'H1', 'W1'}, Island: {'H2'}, Right: set(), Boat: island
Step 2: Left: {'H2', 'W2', 'W1', 'H1'}, Island: set(), Right: set(), Boat: left
Step 3: Left: {'W1', 'H1'}, Island: {'H2', 'W2'}, Right: set(), Boat: island
Step 4: Left: {'W2', 'H1', 'W1'}, Island: {'H2'}, Right: set(), Boat: left
Step 5: Left: {'H1', 'W1'}, Island: {'H2', 'W2'}, Right: set(), Boat: island
Step 6: Left: {'H1', 'W1'}, Island: {'H2', 'W2'}, Right: set(), Boat: left
Step 7: Left: set(), Island: {'H2', 'W2', 'H1', 'W1'}, Right: set(), Boat: island
Step 8: Left: {'H2', 'W2'}, Island: {'W1', 'H1'}, Right: set(), Boat: left
Step 9: Left: {'H2'}, Island: {'W2', 'H1', 'W1'}, Right: set(), Boat: island
Step 10: Left: {'H2', 'H1'}, Island: {'W2', 'W1'}, Right: set(), Boat: left
Step 11: Left: {'H1'}, Island: {'H2', 'W2', 'W1'}, Right: set(), Boat: island
Step 12: Left:

In [7]:
# 3. A* Solution
def heuristic(state: State) -> int:
    """Heuristic function: number of people not on right bank"""
    return len(state.left_bank) + len(state.island)

def astar_solve(initial_state: State, couples: List[Tuple[str, str]]) -> Optional[List[State]]:
    visited = set()
    queue = PriorityQueue()
    queue.put((0, 0, initial_state, [initial_state]))  # (f_score, tiebreaker, state, path)
    tiebreaker = 1
    
    while not queue.empty():
        _, _, current_state, path = queue.get()
        state_str = str(current_state)
        
        if state_str in visited:
            continue
            
        visited.add(state_str)
        
        # Check if goal state
        if not current_state.left_bank and not current_state.island:
            return path
        
        # Try all possible moves
        for move in get_possible_moves(current_state):
            new_state = apply_move(current_state, move)
            if is_valid_state(new_state, couples):
                new_path = path + [new_state]
                f_score = len(new_path) + heuristic(new_state)
                queue.put((f_score, tiebreaker, new_state, new_path))
                tiebreaker += 1
    
    return None

print("\nSolving with A*:")
astar_solution = astar_solve(initial_state, couples)
if astar_solution:
    print(f"Found solution with {len(astar_solution)} steps")
    for i, state in enumerate(astar_solution):
        print(f"Step {i}: {state}")
else:
    print("No solution found with A*")


Solving with A*:
Found solution with 9 steps
Step 0: Left: {'H2', 'W2', 'H1', 'W1'}, Island: set(), Right: set(), Boat: left
Step 1: Left: {'W1', 'H1'}, Island: {'H2', 'W2'}, Right: set(), Boat: island
Step 2: Left: {'H1', 'W1'}, Island: {'H2', 'W2'}, Right: set(), Boat: left
Step 3: Left: set(), Island: {'H2', 'W2', 'H1', 'W1'}, Right: set(), Boat: island
Step 4: Left: set(), Island: {'W1', 'H1'}, Right: {'H2', 'W2'}, Boat: right
Step 5: Left: set(), Island: {'W2', 'H1', 'W1'}, Right: {'H2'}, Boat: island
Step 6: Left: set(), Island: {'H1'}, Right: {'H2', 'W2', 'W1'}, Boat: right
Step 7: Left: set(), Island: {'H2', 'H1'}, Right: {'W2', 'W1'}, Boat: island
Step 8: Left: set(), Island: set(), Right: {'H2', 'W2', 'H1', 'W1'}, Boat: right
